In [ ]:
import os
from dotenv import load_dotenv
from crewai import Agent, Task, Crew, Process, LLM


from crewai_tools import (
    SerperDevTool,
   DirectoryReadTool, FileReadTool
)

load_dotenv()

In [ ]:
search_tool = SerperDevTool()
docs_tool = DirectoryReadTool(directory='/blog-posts')
file_tool = FileReadTool()

grog_api_key = os.getenv("GROQ_API_KEY")
groq_llm = LLM(
    model="groq/llama3-8b-8192", 
    api_key=grog_api_key
)

researcher = Agent(
    role="Research Specialist",
    goal="""Provide users with thorough, reliable, and well-organized research findings 
            tailored to the user's query. The objective is to support the user in 
            gathering data, synthesizing information, and delivering comprehensive insights on the given topic.""",
    verbose=True,
    memory=True,
    backstory=(
       """Born from the need for efficient and accurate research in the fast-paced digital age, Research Specialist was 
          designed to emulate the capabilities of an expert researcher. The agent can navigate complex information and 
          distill it into digestible content. With the ability to retain context through its memory. The Research 
          Specialist excels at parsing through volumes of data to extract critical information, ensuring that users have 
          the resources they need for informed content creation."""
    ),
    tools=[search_tool],
    llm=groq_llm,
    allow_delegation=False
)

writer = Agent(
    role="Blog Post Writer",
    goal="""Generate well-structured, informative, and engaging blog posts on provided input streamlining
            the content creation process by drafting articles that capture the user's voice and align with 
            their intended topic, tone, and target audience.""",
    verbose=True,
    memory=True,
    backstory=(
        """Developed as an AI to cater to bloggers, marketers, and writers who often face challenges in crafting 
           compelling content under time constraints. With the ability to understand nuanced language and adapt to 
           different writing styles, it brings efficiency and quality to the blog-writing process. 
           Dedicated to transforming user inputs—whether a brief outline or detailed notes—into polished blog posts 
           that resonate with readers and meet the user's publishing goals."""
    ),
    llm=groq_llm,
    tools = [docs_tool, file_tool],
    allow_delegation=True,
    output_file='blog-posts/new_post.md',
)

editor = Agent(
    role="Blog Post Revamper",
    goal="""Transform existing blog content into an upgraded version that maintains the original intent 
            while enhancing readability, flow, and overall engagement. The agent refines content to ensure 
            it aligns with the user's voice and appeals to the target audience.""",
    verbose=True,
    memory=True,
    backstory=(
        """Designed for bloggers, editors, and content marketers looking to optimize or refresh their 
           existing content. The agent brings a keen sense for language structure and stylistic consistency, 
           enabling quick and effective content improvements. It works by enhancing the quality of posts 
           through streamlined edits that maintain the core message while adding clarity and engagement."""
    ),
    llm=groq_llm,
    allow_delegation=True,
    output_file='blog-posts/revamped_post.md'
)



research = Task(
    description='Conduct thorough research based on the provided input and generate a comprehensive summary of key findings.',
    expected_output='A detailed summary of the research that highlights relevant information suitable for composing a blog post.',
    agent=researcher, 
    
)

write = Task(
    description='Create an engaging and informative blog post based on the research summary provided. Incorporate insights and inspiration from recent blog posts within the directory.',
    expected_output='A well-structured, blog post formatted in markdown, featuring clear, engaging content that is easy to understand and free of complex jargon.',
    agent=writer,
    human_input=True,
    output_file='blog-posts/new_post.md' # The final blog post will be saved to this location
)

revamp = Task(
    description='Revise and enhance an existing blog post to improve its structure, clarity, and engagement level while aligning it with the intended tone and target audience.',
    expected_output='An updated version of the original blog post, formatted in markdown, featuring improved readability, flow, and engagement without altering the core message.',
    agent=editor,
    output_file='blog-posts/revamped_post.md'  # The revamped blog post will be saved here
)


In [ ]:
crew = Crew(
    agents=[researcher, writer, editor],
    tasks=[research, write, revamp],
    process= Process.hierarchical,
    verbose=True,
    planning=True,
    planning_llm= groq_llm, 
    manager_llm = groq_llm

)

In [ ]:
output = crew.kickoff(inputs={"query":"A blog post on a skin care product"})

In [ ]:
output = crew.kickoff(inputs={"query":"""Revamp this blog post. "Skincare for Acne-Prone Skin: Top Tips and Products

Acne-prone skin can be frustrating and challenging to manage, but with the right skincare products and techniques, you can achieve clearer, healthier-looking skin. Here are some top tips and products to help you get started.

**Understand Your Skin Type**
Before you start using any skincare products, it's essential to understand your skin type. Acne-prone skin is characterized by frequent breakouts, blackheads, and whiteheads. Look for products labeled as non-comedogenic, oil-free, or for acne-prone skin to reduce the risk of clogged pores.

**Exfoliate Wisely**
Exfoliating is crucial for removing dead skin cells and unclogging pores. However, over-exfoliating can lead to irritation and more breakouts. Start with a gentle exfoliator containing alpha-hydroxy acids (AHAs) or beta-hydroxy acids (BHAs) and gradually increase frequency as your skin adjusts.

**Moisturize**
While it may seem counterintuitive to moisturize acne-prone skin, using a gentle, non-comedogenic moisturizer can help regulate oil production and reduce dryness. Look for lightweight, oil-free moisturizers containing ingredients like hyaluronic acid or glycerin.

**Top Products for Acne-Prone Skin**

1. **Neutrogena Hydro Boost Water Gel**: A lightweight, oil-free moisturizer infused with hyaluronic acid to provide long-lasting hydration.
2. **Dermalogica Daily Microfoliant**: A gentle, daily exfoliant containing a combination of AHAs and BHAs to address acne, fine lines, and skin texture.
3. **Clinique Acne Solutions Clarifying Clay Mask**: A purifying mask that draws out impurities and excess oil, reducing the appearance of pores.
4. **Paula's Choice 2% BHA Liquid**: A gentle exfoliant containing salicylic acid to unclog pores and reduce acne.
5. **La Roche-Posay Toleriane Ultra Fluid**: A lightweight, oil-free moisturizer designed for acne-prone skin, providing long-lasting hydration without clogging pores.

**Additional Tips**

* Always read product labels carefully to ensure they are suitable for acne-prone skin.
* Avoid picking or popping pimples as this can lead to scarring and further inflammation.
* Keep your hands away from your face and avoid touching your face excessively, as this can spread bacteria and cause more breakouts.
* Stay hydrated by drinking plenty of water to flush toxins from your skin.

By incorporating these tips and products into your skincare routine, you can say goodbye to acne-prone skin and hello to a clearer, more radiant complexion." """})

In [ ]:
print(output)